# 자연어 전처리

In [1]:
stopwords=[]

In [2]:
# 토크나이징 함수 정의
from konlpy.tag import Okt
okt= Okt()
def tokenizer(raw, pos=["Noun","Alpha","Verb","Number","Foreign"], stopword=stopwords):
    return [
        word for word, tag in okt.pos(
            raw, 
            norm=True,   # normalize 그랰ㅋㅏ -> 그래ㅋㅋ
            stem=True    # stemming 바뀌나->바뀌다
            )
            if len(word) > 1 and tag in pos and word not in stopword

    ]

In [3]:
import pandas as pd

In [4]:
get_data = pd.read_csv("https://bit.ly/seoul-120-text-csv")
get_data

,번호,분류,제목,내용,내용번호
0,2645,복지,아빠 육아휴직 장려금,아빠 육아휴직 장려금 업무개요 남성근로자의 육아휴직을 장려하고 양육에 따른 경...,23522464
1,2644,경제,[서울산업진흥원] 서울메이드란?,서울산업진흥원 서울메이드란 서울의 감성을 담은 다양하고 새로운 경험을 제공하기 위해...,23194045
2,2643,환경,(강북구) 정비중,강북구 정비중 업무개요 투명 폐트병을 교환보상하므로 수거율을 높이고 폐기물을 감...,23032485
3,2642,복지,"광진맘택시 운영(임산부,영아 양육가정 전용 택시)",광진맘택시 운영임산부영아 양육가정 전용 택시 업무개요 교통약자인 임산부와 영아가정...,22904492
4,2641,복지,마포 뇌병변장애인 비전센터,마포 뇌병변장애인 비전센터 마포뇌병변장애인 비전센터 운영 구분 내용 목적 학...,22477798
...,...,...,...,...,...
2640,5,주택도시계획,이미 지정된 재정비촉진지구의 인접지를 추가 확장하고 싶은데 어떻게 해야 하나요?,이미 지정된 재정비촉진지구의 인접지를 추가 확장하고 싶은데 어떻게 해야 하나요 관할...,2894353
2641,4,환경,수돗물의 톤당 원가는?,수돗물의 톤당 원가는우리시 년 톤당 원가는 원 입니다 우리시 년 톤당 요금은 원 입니다,19353862
2642,3,복지,출산휴가후 집근처에 안심하고 맡길 수 있는 어린이집이 있는지?,출산휴가후 집근처에 안심하고 맡길 수 있는 어린이집이 있는지 서울시 육아종합지원...,19699584
2643,2,교통,자동차검사장을 지정받고자 하는데 어떻게 해야 하나요?,자동차검사장을 지정받고자 하는데 어떻게 해야 하나요 자동차종합정비업체 소형자동차정...,2898293


In [5]:
# tokenizing
from sklearn.feature_extraction.text import TfidfVectorizer
tfidVectorizer = TfidfVectorizer(tokenizer=tokenizer)

In [6]:
features = tfidVectorizer.fit_transform(get_data['내용'])
features.toarray()[:2]

/usr/local/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

# 유사도 확인

In [7]:
from sklearn.metrics.pairwise import cosine_similarity

In [18]:
features.toarray()[1772:1773]

array([[0., 0., 0., ..., 0., 0., 0.]])

In [9]:
result = cosine_similarity(features.toarray()[1772:1773], features.toarray())
len(result)

1

In [10]:
result

array([[1.        , 0.00570703, 0.02195183, ..., 0.0209912 , 0.01709235,
        0.01261128]])

In [11]:
get_data['유사도'] = result[0]

In [12]:
get_data

,번호,분류,제목,내용,내용번호,유사도
0,2645,복지,아빠 육아휴직 장려금,아빠 육아휴직 장려금 업무개요 남성근로자의 육아휴직을 장려하고 양육에 따른 경...,23522464,1.000000
1,2644,경제,[서울산업진흥원] 서울메이드란?,서울산업진흥원 서울메이드란 서울의 감성을 담은 다양하고 새로운 경험을 제공하기 위해...,23194045,0.005707
2,2643,환경,(강북구) 정비중,강북구 정비중 업무개요 투명 폐트병을 교환보상하므로 수거율을 높이고 폐기물을 감...,23032485,0.021952
3,2642,복지,"광진맘택시 운영(임산부,영아 양육가정 전용 택시)",광진맘택시 운영임산부영아 양육가정 전용 택시 업무개요 교통약자인 임산부와 영아가정...,22904492,0.108691
4,2641,복지,마포 뇌병변장애인 비전센터,마포 뇌병변장애인 비전센터 마포뇌병변장애인 비전센터 운영 구분 내용 목적 학...,22477798,0.033441
...,...,...,...,...,...,...
2640,5,주택도시계획,이미 지정된 재정비촉진지구의 인접지를 추가 확장하고 싶은데 어떻게 해야 하나요?,이미 지정된 재정비촉진지구의 인접지를 추가 확장하고 싶은데 어떻게 해야 하나요 관할...,2894353,0.026592
2641,4,환경,수돗물의 톤당 원가는?,수돗물의 톤당 원가는우리시 년 톤당 원가는 원 입니다 우리시 년 톤당 요금은 원 입니다,19353862,0.000000
2642,3,복지,출산휴가후 집근처에 안심하고 맡길 수 있는 어린이집이 있는지?,출산휴가후 집근처에 안심하고 맡길 수 있는 어린이집이 있는지 서울시 육아종합지원...,19699584,0.020991
2643,2,교통,자동차검사장을 지정받고자 하는데 어떻게 해야 하나요?,자동차검사장을 지정받고자 하는데 어떻게 해야 하나요 자동차종합정비업체 소형자동차정...,2898293,0.017092


In [14]:
get_data.sort_values(by=['유사도'], ascending=False)[:5]

,번호,분류,제목,내용,내용번호,유사도
0,2645,복지,아빠 육아휴직 장려금,아빠 육아휴직 장려금 업무개요 남성근로자의 육아휴직을 장려하고 양육에 따른 경...,23522464,1.000000
1386,1259,행정,육아휴직수당은 얼마인가요?,육아휴직수당은 얼마인가요자녀을 양육하기 위하여 일 이상 휴직한 남녀공무원이 대상이며...,2896657,0.259316
72,2573,복지,"아이조아 아빠교실, 아이조아 놀이교실",아이조아 아빠교실 아이조아 놀이교실 업무개요 다양한 교육과 체...,4369620,0.155758
35,2610,행정,[시ㆍ구정외 타기관 관련 상담] 고용노동부 [일자리 안정자금],시구정외 타기관 관련 상담 고용노동부 일자리 안정자금 최저임금 해결사 일자리 안정자...,14425328,0.133478
914,1731,복지,한우리정보문화센터에 관해서 알고 싶습니다,한우리정보문화센터에 관해서 알고 싶습니다ㅎ 서초구에서 운영하는 장애인 복지관 홈페...,2895013,0.133133
